In [2]:
import eikon as ek  # the Eikon Python wrapper package
import pandas as pd  # pandas

# Set the app key
ek.set_app_key('ed38bad70d5349ab8f74c842ba1fc15d1964a885')

# Get the data for different countries

# United States
# set the file path
file_path_us= r'C:\Users\cm12c\OneDrive\Documents\dashboard_data.xlsx'
# get the data

pmi_us= ek.get_timeseries('USPMI=ECI', start_date='2000-01-01', interval='monthly')
confidence_us= ek.get_timeseries('USCONC=ECI', start_date='2000-01-01', interval='monthly')
pmi_ca= ek.get_timeseries('CAIPMI=ECI', start_date='2000-01-01', interval='monthly')
confidence_ca= ek.get_timeseries('aCACONCF', start_date='2000-01-01', interval='monthly')


# write the file to excel
writer_us = pd.ExcelWriter(file_path_us, engine='xlsxwriter')
# Write each data set to a separate sheet

pmi_us.to_excel(writer_us, sheet_name='pmi_us')
confidence_us.to_excel(writer_us, sheet_name='confidence_us')
pmi_ca.to_excel(writer_us, sheet_name='pmi_ca')
confidence_ca.to_excel(writer_us, sheet_name='confidence_ca')


# Save the Excel file and close the Pandas Excel writer
writer_us.close()

In [19]:
pe_us=ek.get_data(['.HSCE'],['TR.Index_PE_RTRS.date','TR.Index_PE_RTRS'],{"SDate":'0D',"EDate":'-25CY'})
pe_us = pd.DataFrame(pe_us[0])

In [20]:
pe_us

,Instrument,Date,Calculated PE Ratio
0,.HSCE,2023-09-22T00:00:00Z,8.73235
1,.HSCE,2023-09-21T00:00:00Z,8.50504
2,.HSCE,2023-09-20T00:00:00Z,8.61365
3,.HSCE,2023-09-19T00:00:00Z,8.66339
4,.HSCE,2023-09-18T00:00:00Z,8.62477
...,...,...,...
5683,.HSCE,2000-10-27T00:00:00Z,7.802846
5684,.HSCE,2000-10-24T00:00:00Z,7.618984
5685,.HSCE,2000-10-20T00:00:00Z,7.502422
5686,.HSCE,2000-03-31T00:00:00Z,6.360256


In [119]:
from graphs import *
from arch import arch_model
from scipy.stats import mstats
from datetime import datetime as dt
import numpy as np
import plotly.graph_objs as go
import plotly.subplots as sp

In [120]:
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 600,
    'width': 800,
    'scale': 7  }} # Multiply title/legend/axis/canvas sizes by this factor

In [121]:
unrate_usa()
cpi_usa()
fedfunds_usa()
vix()
spreads_usa()
savings_usa()
sp_tsx()
sp_500()
msci_china()

In [122]:
sp_tsx()

In [123]:
def forecast_volatility(df:pd.DataFrame,winsorize_limits:float,window_size=1000)->pd.Series:
    """
    Function
    ---------- 
    Calculate the daily forecasted volatilty with a GARCH(1,1) model
    
    Parameters
    ----------
    df : a DataFrame of the daily return of the index
    winsorize_limits : the limits of the winsorization ex: [0.05,0.05]
    test_size : the size of the test set. The rest will be train set ex: 0.5
    
    Returns
    ------
    Returns a Series of the forecasted volatility
    """
    returns = df.loc[:,df.columns[0]].values*100
    winsorized_returns = mstats.winsorize(returns, limits = winsorize_limits)
    
    window_size = window_size
    forecasts_std = []
    for i in range(window_size, len(returns)):
        window = winsorized_returns[i-window_size:i]
        model = arch_model(window, vol = 'GARCH', p = 1, q = 1).fit(disp='off')
        forecast = model.forecast(horizon = 1,reindex = False)
        forecasts_std.append(np.sqrt(forecast.variance.values[-1,-1]))
    forecasts_std = pd.DataFrame(forecasts_std, index = df.index[window_size:])
    forecasts_std.index.name = 'Date'
    
    return forecasts_std

In [124]:
df_prix = yf.download('MCHI', '2000-01-01' , dt.today(),progress=False)
df_prix = df_prix[['Adj Close']] 
df_prix = df_prix.rename(columns={'Adj Close': 'MCHI'})
df = df_prix.pct_change().dropna() 


In [125]:
forecast_vol_china=forecast_volatility(df, [0.05,0.05], window_size=1000)
forecast_vol_china

,0
Date,
2015-03-25,1.113338
2015-03-26,1.119840
2015-03-27,1.099763
2015-03-30,1.117412
2015-03-31,1.202442
...,...
2023-09-18,1.415347
2023-09-19,1.383603
2023-09-20,1.367113


In [126]:
df_prix = df_prix.iloc[1001:]
fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.7, 0.3])

returns_trace = go.Scatter(x=df_prix.index, y=df_prix.iloc[:,0],line_color='#00334E', name='Prix')
fig.add_trace(returns_trace, row=1, col=1)

volatility_trace = go.Scatter(x=forecast_vol_china.index, y=forecast_vol_china.iloc[:,0],line_color='#ED1C26', name='Volatilité prévue')
fig.add_trace(volatility_trace, row=2, col=1)

# Set axis labels and titles
fig.update_xaxes(title_text="Date",row=2, col=1)
fig.update_yaxes(title_text="Prix", row=1, col=1)
fig.update_yaxes(title_text="Volatilité prévue", row=2, col=1)

# Configure layout
fig.update_layout(title={'text': "Prix de l'actif vs volatilité prévue",'x': 0.5,'xanchor': 'center'},template="simple_white",showlegend=False)

quintiles = np.percentile(forecast_vol_china.iloc[:, 0], [20, 80])
for q in quintiles:
    fig.add_trace(go.Scatter(x=[forecast_vol_china.index.min(),forecast_vol_china.index.max()], y=[q,q],line_color='grey',mode='lines',line_dash='dash'),row=2, col=1)


fig.write_html("graph/china_vol.html",config=config)